In [1]:
import pickle
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [2]:
# walk the experiment directory
all_experiments = {}
for root, dirs, files in os.walk('../experiments/experiments'):
    experiment = defaultdict(dict)
    for f in files:
        key = '/'.join(root.split('/')[1:-1])    
        trial = root.split('/')[-1]
        if f == "options.pkl":
            opts = vars(pickle.load(open(os.path.join(root, f), 'rb')))
            experiment = all_experiments.get(key, defaultdict(dict))
            experiment[trial]['opts'] = opts
            all_experiments[key] = experiment
        elif f.startswith("event"):
            experiment = all_experiments.get(key, defaultdict(dict))
            experiment[trial]['event_accumulator'] = EventAccumulator(os.path.join(root, f))
            all_experiments[key] = experiment


In [3]:
def filter_experiments(experiments):
    accumulators = []
    for exp in experiments.values():
        if exp.values()[0].get('event_accumulator') is not None:
            accumulators.append(exp)
    return accumulators

In [4]:
experiments = filter_experiments(all_experiments)

In [57]:
def make_trial_df(trial):
    scalar_tags = trial['event_accumulator'].Reload().Tags()['scalars']
    opt_tags = trial['opts'].keys()
    my_tags = ['auc_train', 'auc_valid', 'auc_test', 'acc_train', 'acc_valid', 'acc_test']
    df = pd.DataFrame(columns=opt_tags + my_tags)
    event_accumulator = trial['event_accumulator'].Reload()
    for tag in my_tags:
        # scalars = np.array(map(lambda x: x.value, events))
        print len(event_accumulator.Scalars(tag))
        df.loc[0, tag] = event_accumulator.Scalars(tag)[0].value
    for tag in opt_tags:
        df.loc[0, tag] = trial['opts'][tag]
    return df

In [ ]:
raw_df = None
for exp in experiments:
    exp_df = None
    for key, trial in exp.iteritems():
        if exp_df is not None:
            exp_df = exp_df.append(make_trial_df(trial))
        else:
            exp_df = make_trial_df(trial)
    if raw_df is not None:   
        raw_df = raw_df.append(exp_df)
    else:
        raw_df = exp_df

    

> <ipython-input-58-2018531d84ea>(6)<module>()->None
-> if exp_df is not None:
(Pdb) n
> <ipython-input-58-2018531d84ea>(9)<module>()->None
-> exp_df = make_trial_df(trial)
(Pdb) c
1
1
1
1
1
1
> <ipython-input-58-2018531d84ea>(5)<module>()->None
-> import pdb; pdb.set_trace()
(Pdb) c
1
1
1
1
1
1
> <ipython-input-58-2018531d84ea>(6)<module>()->None
-> if exp_df is not None:
(Pdb) c
1
1
1
1
1
1
> <ipython-input-58-2018531d84ea>(5)<module>()->None
-> import pdb; pdb.set_trace()
(Pdb) c
1
1
1
1
1
1
> <ipython-input-58-2018531d84ea>(6)<module>()->None
-> if exp_df is not None:
(Pdb) c
9
9
8
9
9
8
> <ipython-input-58-2018531d84ea>(5)<module>()->None
-> import pdb; pdb.set_trace()
(Pdb) l
  1  	raw_df = None
  2  	for exp in experiments:
  3  	    exp_df = None
  4  	    for key, trial in exp.iteritems():
  5  ->	        import pdb; pdb.set_trace()
  6  	        if exp_df is not None:
  7  	            exp_df = exp_df.append(make_trial_df(trial))
  8  	        else:
  9  	            exp_df =

In [52]:
print raw_df

Empty DataFrame
Columns: [acc_test, acc_train, acc_valid, add_connectivity, add_self, approx_nb_edges, attention_layer, auc_test, auc_train, auc_valid, batch_size, center, clinical_file, clinical_label, cuda, dataset, disconnected, dropout, epoch, extra_cn, extra_ucn, graph, l1_loss_lambda, load_checkpoint, load_folder, log, lr, model, model_reg_lambda, momentum, name, nb_attention_head, nb_class, nb_examples, nb_nodes, nb_per_class, neighborhood, norm_adj, num_channel, num_layer, percentile, pool_graph, scale_free, seed, semi_mse_lambda, size_perc, tensorboard_dir, train_ratio, training_mode, trial_number, use_emb, use_gate, weight_decay]
Index: []

[0 rows x 53 columns]


In [37]:
def take_mean(events):
    print events
    mean = np.array([e.value() for e in events]).mean()
    import pdb; pdb.set_trace()
    print mean

In [ ]:
e = trial['event_accumulator']
auc = e.Scalars('auc_train')
take_mean(auc)
import pdb; pdb.set_trace()

--Call--
> /Tmp/lisa/os_v5/anaconda/lib/python2.7/site-packages/IPython/core/displayhook.py(236)__call__()
-> def __call__(self, result=None):
(Pdb) auc
*** NameError: name 'auc' is not defined
(Pdb) print auc
*** NameError: name 'auc' is not defined
(Pdb) print e
*** NameError: name 'e' is not defined
(Pdb) q


In [ ]:
trial = experiments.values()[0].values()[0]
scalar_tags = trial['event_accumulator'].Reload().Tags()['scalars']
opt_tags = trial['opts'].keys()
df = pd.DataFrame(columns=opt_tags + scalar_tags)
exp_df = pd.DataFrame(columns=opt_tags + scalar_tags)
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        import pdb; pdb.set_trace()

        event_accumulator = trial['event_accumulator'].Reload()
        for tag in scalar_tags:
            events = event_accumulator.Scalars(tag)
            scalars = np.array(map(lambda x: x.value, events))
            df.loc[:, tag] = scalars
        for tag in opt_tags:
            df.loc[:, tag] = [trial['opts'][tag]]
        exp_df = exp_df.append(df)


In [ ]:
for path, experiment in experiments.iteritems():
    for number, trial in experiment.iteritems():
        print trial